# SQL Assignment: Setting Up the Database Connection
In this assignment, you will learn how to connect to a database using `SQLAlchemy` and environment variables. Follow the steps below carefully to set up the connection to your database.

### Install all libraries
Let's start by installing all the libraries listed in `requirements.txt`:

In [1]:
# TODO: Run this cell to install libraries listed in `requirements.txt`
!pip install -r ../requirements.txt

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


### Create a `.env` file
1. In the root directory of your project, you should see a file named `.env.example`. This file provides a template for setting up your database connection variables.

2. Duplicate the `.env.example` file and rename the copy to `.env`. You can do this in your terminal or file explorer.
    - In the terminal, run the following command:

        ```python
        cp .env.example .env
        ```

3. Open the `.env` file and modify the following line:

    ```python
    DB_NAME = '<your database name>'
    ```

    Replace `<your database name>` with the name of your custom database. For example:

    ```python
    DB_NAME = 'my_database'
    ```

4. Save the changes to the `.env` file.

### Verify Your `.env` File Setup
1. Run the following code cell to confirm that the environment variables in your `.env` file are being loaded correctly:

In [2]:
# TODO: Run this cell to import the necessary libraries and load your variables you set in your .env file
import os
from sqlalchemy import create_engine, inspect
import pandas as pd
from dotenv import load_dotenv

# Load the .env file variables
# The output of this command should be `True`
# If the output is `False`, that means you did not create a valid .env file in the root directory of this project
load_dotenv()

True

2. If your `.env` file is set up correctly, the command should return `True`. If it returns `False`, check the following:

    - Ensure the `.env` file is located in the **root directory** of your project.

    - Double-check that you replaced `<your database name>` with your actual database name.

    - Make sure there are no syntax errors in the `.env` file (e.g., no extra spaces or missing values).

3. You can also validate and inspect your environment variables by running the following code cell:

In [3]:
# Inspect the environment variables
print(os.getenv('DB_USER'))
print(os.getenv('DB_HOST'))
print(os.getenv('DB_NAME'))
print(os.getenv('DB_PASSWORD'))

gitpod
localhost
gabe
postgres


### Connecting to the Database
Now that you have set up your `.env` file and confirmed the environment variables are loaded correctly, it’s time to establish a connection to your database using `SQLAlchemy`'s `create_engine` function. 

1. Run the following code to establish a connection:

In [30]:
# 1) Connect to the database here using the SQLAlchemy's `create_engine`` function
# A "connection string" is basically a string containing all database credentials together.
connection_string = f"postgresql://{os.getenv('DB_USER')}:{os.getenv('DB_PASSWORD')}@{os.getenv('DB_HOST')}/{os.getenv('DB_NAME')}"
engine = create_engine(connection_string)
engine.connect()
print("Starting the connection...")

Starting the connection...


2. **Handling the `... FATAL:  database "<DB_NAME>" does not exist` Error**

    - If the connection attempt throws the following error:

        ```python
        ... FATAL:  database "<DB_NAME>" does not exist
        ```

        This means your database hasn't been created yet. To fix this, follow theses steps:

    - Open your **terminal**

    - Run the following command to create your database, making sure to replace `<DB_USER>` and `<DB_NAME>` with the values you set in your `.env` file:

        ```python
        createdb -h localhost -U <DB_USER> <DB_NAME>
        ```

        For example, if you set `DB_USER` to `gitpod` and `DB_NAME` to `marisols_database`, the command would be:

        ```python
        createdb -h localhost -U gitpod marisols_database
        ```

    - If prompted, enter the password you set in your `.env` file for the `DB_USER`

    - Rerun the code cell above. If successfull, the output will be `Starting the connection...`

### Inspecting the Database for Tables
Now that you've connected to the database, let's check whether any tables currently exist. Since you haven't created any tables yet, the output should be an empty list.

`SQLAlchemy` provides a helpful function called `inspect` to examine the database. In this step, you'll use it to list all existing tables in the database.

Run the following code to inspect the database:

In [5]:
# We can inspect our database by creating an inspector object
inspector = inspect(engine)

# Get the table names
table_names = inspector.get_table_names()

# Print the table names
print(f"Existing tables in the database: {table_names}")

Existing tables in the database: []


Since the database is currently empty (you haven’t created any tables yet).

### Creating Tables in the Database
We need to create four tables: `publishers`, `authors`, `books`, and `book_authors`, with foreign key relationships between them. These relationships will help organize the data logically and maintain integrity.

- **Check if a table exists**: To avoid errors from trying to create a table that already exists, we use `CREATE TABLE IF NOT EXISTS`. This ensures that the table is created only if it doesn't already exist in the database.

Run the following code to create the tables:

In [6]:
from sqlalchemy import text

# Connect to the database
with engine.begin() as connection:
    # Execute the SQL statements to create tables
    connection.execute(text("""
    CREATE TABLE IF NOT EXISTS publishers(
        publisher_id INT NOT NULL,
        name VARCHAR(255) NOT NULL,
        PRIMARY KEY(publisher_id)
    );

    CREATE TABLE IF NOT EXISTS authors(
        author_id INT NOT NULL,
        first_name VARCHAR(100) NOT NULL,
        middle_name VARCHAR(50) NULL,
        last_name VARCHAR(100) NULL,
        PRIMARY KEY(author_id)
    );

    CREATE TABLE IF NOT EXISTS books(
        book_id INT NOT NULL,
        title VARCHAR(255) NOT NULL,
        total_pages INT NULL,
        rating DECIMAL(4, 2) NULL,
        isbn VARCHAR(13) NULL,
        published_date DATE,
        publisher_id INT NULL,
        PRIMARY KEY(book_id),
        CONSTRAINT fk_publisher FOREIGN KEY(publisher_id) REFERENCES publishers(publisher_id)
    );

    CREATE TABLE IF NOT EXISTS book_authors (
        book_id INT NOT NULL,
        author_id INT NOT NULL,
        PRIMARY KEY(book_id, author_id),
        CONSTRAINT fk_book FOREIGN KEY(book_id) REFERENCES books(book_id) ON DELETE CASCADE,
        CONSTRAINT fk_author FOREIGN KEY(author_id) REFERENCES authors(author_id) ON DELETE CASCADE
    );
    """))


After running the SQL statements, you can re-inspect the database to confirm that the tables were created successfully:

In [7]:
# Let's inspect out database again to confirm that the tables were created
inspector = inspect(engine)

# Get the table names
table_names = inspector.get_table_names()

# Print the table names
print(f"Existing tables in the database: {table_names}")

Existing tables in the database: ['publishers', 'books', 'book_authors', 'authors']


### Inserting Data into the Tables
Now you will insert sample data into the tables you created earlier: `publishers`, `authors`, `books`, and `book_authors`. This will allow you to work with real data in your database for further queries and analyses.

In [8]:
# Execute the SQL sentences to insert data into tables
with engine.begin() as connection:
    connection.execute(text("""
    INSERT INTO publishers(publisher_id, name) VALUES (1, 'O Reilly Media');
    INSERT INTO publishers(publisher_id, name) VALUES (2, 'A Book Apart');
    INSERT INTO publishers(publisher_id, name) VALUES (3, 'A K PETERS');
    INSERT INTO publishers(publisher_id, name) VALUES (4, 'Academic Press');
    INSERT INTO publishers(publisher_id, name) VALUES (5, 'Addison Wesley');
    INSERT INTO publishers(publisher_id, name) VALUES (6, 'Albert&Sweigart');
    INSERT INTO publishers(publisher_id, name) VALUES (7, 'Alfred A. Knopf');

    INSERT INTO authors (author_id, first_name, middle_name, last_name) VALUES (1, 'Merritt', null, 'Eric');
    INSERT INTO authors (author_id, first_name, middle_name, last_name) VALUES (2, 'Linda', null, 'Mui');
    INSERT INTO authors (author_id, first_name, middle_name, last_name) VALUES (3, 'Alecos', null, 'Papadatos');
    INSERT INTO authors (author_id, first_name, middle_name, last_name) VALUES (4, 'Anthony', null, 'Molinaro');
    INSERT INTO authors (author_id, first_name, middle_name, last_name) VALUES (5, 'David', null, 'Cronin');
    INSERT INTO authors (author_id, first_name, middle_name, last_name) VALUES (6, 'Richard', null, 'Blum');
    INSERT INTO authors (author_id, first_name, middle_name, last_name) VALUES (7, 'Yuval', 'Noah', 'Harari');
    INSERT INTO authors (author_id, first_name, middle_name, last_name) VALUES (8, 'Paul', null, 'Albitz');

    INSERT INTO books (book_id, title, total_pages, rating, isbn, published_date, publisher_id) VALUES (1, 'Lean Software Development: An Agile Toolkit', 240, 4.17, '9780320000000', '2003-05-18', 5);
    INSERT INTO books (book_id, title, total_pages, rating, isbn, published_date, publisher_id) VALUES (2, 'Facing the Intelligence Explosion', 91, 3.87, null, '2013-02-01', 7);
    INSERT INTO books (book_id, title, total_pages, rating, isbn, published_date, publisher_id) VALUES (3, 'Scala in Action', 419, 3.74, '9781940000000', '2013-04-10', 1);
    INSERT INTO books (book_id, title, total_pages, rating, isbn, published_date, publisher_id) VALUES (4, 'Patterns of Software: Tales from the Software Community', 256, 3.84, '9780200000000', '1996-08-15', 1);
    INSERT INTO books (book_id, title, total_pages, rating, isbn, published_date, publisher_id) VALUES (5, 'Anatomy Of LISP', 446, 4.43, '9780070000000', '1978-01-01', 3);
    INSERT INTO books (book_id, title, total_pages, rating, isbn, published_date, publisher_id) VALUES (6, 'Computing machinery and intelligence', 24, 4.17, null, '2009-03-22', 4);
    INSERT INTO books (book_id, title, total_pages, rating, isbn, published_date, publisher_id) VALUES (7, 'XML: Visual QuickStart Guide', 269, 3.66, '9780320000000', '2009-01-01', 5);
    INSERT INTO books (book_id, title, total_pages, rating, isbn, published_date, publisher_id) VALUES (8, 'SQL Cookbook', 595, 3.95, '9780600000000', '2005-12-01', 7);
    INSERT INTO books (book_id, title, total_pages, rating, isbn, published_date, publisher_id) VALUES (9, 'The Apollo Guidance Computer: Architecture And Operation (Springer Praxis Books / Space Exploration)', 439, 4.29, '9781440000000', '2010-07-01', 6);
    INSERT INTO books (book_id, title, total_pages, rating, isbn, published_date, publisher_id) VALUES (10, 'Minds and Computers: An Introduction to the Philosophy of Artificial Intelligence', 222, 3.54, '9780750000000', '2007-02-13', 7);

    INSERT INTO book_authors (book_id, author_id) VALUES (1, 1);
    INSERT INTO book_authors (book_id, author_id) VALUES (2, 8);
    INSERT INTO book_authors (book_id, author_id) VALUES (3, 7);
    INSERT INTO book_authors (book_id, author_id) VALUES (4, 6);
    INSERT INTO book_authors (book_id, author_id) VALUES (5, 5);
    INSERT INTO book_authors (book_id, author_id) VALUES (6, 4);
    INSERT INTO book_authors (book_id, author_id) VALUES (7, 3);
    INSERT INTO book_authors (book_id, author_id) VALUES (8, 2);
    INSERT INTO book_authors (book_id, author_id) VALUES (9, 4);
    INSERT INTO book_authors (book_id, author_id) VALUES (10, 1);
    """))

### Load SQL Tables into Pandas DataFrames
We will use the `pandas` library to read the data from the SQL tables we created and inserted data into in the previous step. The `pd.read_sql` function allows us to execute SQL queries and return the results as pandas DataFrames.

This is useful for analyzing and manipulating the data programmatically. Below are the commands to load each table into its respective DataFrame:

In [9]:
# Use pandas to print one of the tables as dataframe using read_sql function
# Here we are reading the publishers table as a pandas dataframe
publishers_df = pd.read_sql("SELECT * FROM publishers;", engine)
publishers_df

,publisher_id,name
0,1,O Reilly Media
1,2,A Book Apart
2,3,A K PETERS
3,4,Academic Press
4,5,Addison Wesley
5,6,Albert&Sweigart
6,7,Alfred A. Knopf


In [10]:
# authors table
authors_df = pd.read_sql("SELECT * FROM authors;", engine)
authors_df

,author_id,first_name,middle_name,last_name
0,1,Merritt,None,Eric
1,2,Linda,None,Mui
2,3,Alecos,None,Papadatos
3,4,Anthony,None,Molinaro
4,5,David,None,Cronin
5,6,Richard,None,Blum
6,7,Yuval,Noah,Harari
7,8,Paul,None,Albitz


In [11]:
# books table
books_df = pd.read_sql("SELECT * FROM books;", engine)
books_df

,book_id,title,total_pages,rating,isbn,published_date,publisher_id
0,1,Lean Software Development: An Agile Toolkit,240,4.17,9780320000000,2003-05-18,5
1,2,Facing the Intelligence Explosion,91,3.87,None,2013-02-01,7
2,3,Scala in Action,419,3.74,9781940000000,2013-04-10,1
3,4,Patterns of Software: Tales from the Software ...,256,3.84,9780200000000,1996-08-15,1
4,5,Anatomy Of LISP,446,4.43,9780070000000,1978-01-01,3
5,6,Computing machinery and intelligence,24,4.17,None,2009-03-22,4
6,7,XML: Visual QuickStart Guide,269,3.66,9780320000000,2009-01-01,5
7,8,SQL Cookbook,595,3.95,9780600000000,2005-12-01,7
8,9,The Apollo Guidance Computer: Architecture And...,439,4.29,9781440000000,2010-07-01,6
9,10,Minds and Computers: An Introduction to the Ph...,222,3.54,9780750000000,2007-02-13,7


In [12]:
# book_authors table
book_authors_df = pd.read_sql("SELECT * FROM book_authors;", engine)
book_authors_df

,book_id,author_id
0,1,1
1,2,8
2,3,7
3,4,6
4,5,5
5,6,4
6,7,3
7,8,2
8,9,4
9,10,1


## SQL Data Exploration and Analysis
In this section, you will complete 15 exercises that require you to write SQL queries to extract insights from the `publishers`, `authors`, `books`, and `book_authors` tables.

Read each prompt carefully, then write and execute the appropriate SQL query using the provided SQLAlchemy `engine`. Aim to complete all exercises to enhance your SQL skills and deepen your understanding of data manipulation.

In [13]:
# 1. Write a SQL query to select all columns and rows from the `books` table. 
# (Hint: Use `*`)
all_books = pd.read_sql("SELECT * FROM books", engine)
all_books

,book_id,title,total_pages,rating,isbn,published_date,publisher_id
0,1,Lean Software Development: An Agile Toolkit,240,4.17,9780320000000,2003-05-18,5
1,2,Facing the Intelligence Explosion,91,3.87,None,2013-02-01,7
2,3,Scala in Action,419,3.74,9781940000000,2013-04-10,1
3,4,Patterns of Software: Tales from the Software ...,256,3.84,9780200000000,1996-08-15,1
4,5,Anatomy Of LISP,446,4.43,9780070000000,1978-01-01,3
5,6,Computing machinery and intelligence,24,4.17,None,2009-03-22,4
6,7,XML: Visual QuickStart Guide,269,3.66,9780320000000,2009-01-01,5
7,8,SQL Cookbook,595,3.95,9780600000000,2005-12-01,7
8,9,The Apollo Guidance Computer: Architecture And...,439,4.29,9781440000000,2010-07-01,6
9,10,Minds and Computers: An Introduction to the Ph...,222,3.54,9780750000000,2007-02-13,7


In [14]:
# 2. Write a SQL query to select the first 3 rows from the `books` table. 
# (Hint: Use `LIMIT n` where n is the number of rows)
top_three_rows = pd.read_sql("SELECT * FROM books LIMIT 3;", engine)
top_three_rows

,book_id,title,total_pages,rating,isbn,published_date,publisher_id
0,1,Lean Software Development: An Agile Toolkit,240,4.17,9780320000000,2003-05-18,5
1,2,Facing the Intelligence Explosion,91,3.87,None,2013-02-01,7
2,3,Scala in Action,419,3.74,9781940000000,2013-04-10,1


In [15]:
# 3. Write a SQL query to select only the `first_name` column of the `authors` table`.
authors_first_names = pd.read_sql("SELECT first_name FROM authors", engine)
authors_first_names

,first_name
0,Merritt
1,Linda
2,Alecos
3,Anthony
4,David
5,Richard
6,Yuval
7,Paul


In [16]:
# 4. Write a SQL query to to select only the `title` column of the `books` table`.
book_titles = pd.read_sql("SELECT first_name FROM authors;", engine)
book_titles

,first_name
0,Merritt
1,Linda
2,Alecos
3,Anthony
4,David
5,Richard
6,Yuval
7,Paul


In [17]:
# 5. Write a SQL query to select only the `name` column of the `publishers` table`.
publishers = pd.read_sql("SELECT name FROM publishers", engine)
publishers

,name
0,O Reilly Media
1,A Book Apart
2,A K PETERS
3,Academic Press
4,Addison Wesley
5,Albert&Sweigart
6,Alfred A. Knopf


In [18]:
# 6. Write a SQL query to select the `title` and `published_date` columns of the `books` table`.
book_dates = pd.read_sql("SELECT published_date FROM books", engine)
book_dates

,published_date
0,2003-05-18
1,2013-02-01
2,2013-04-10
3,1996-08-15
4,1978-01-01
5,2009-03-22
6,2009-01-01
7,2005-12-01
8,2010-07-01
9,2007-02-13


In [19]:
# 7. Write a SQL query to count the number of rows listed in the `authors` table. 
# (Hint: Use the `COUNT()` command)
row_count = pd.read_sql("SELECT COUNT(*) FROM authors", engine)
row_count

,count
0,8


In [20]:
# 8. Write a SQL query to find the sum of `total_pages` using the `books` table. 
# (Hint: Use the `SUM()` command)
sum_total_pages = pd.read_sql("SELECT SUM(total_pages) FROM books", engine)
sum_total_pages

,sum
0,3001


In [21]:
# 9. Write a SQL query to find the average book `rating` using the `books` table. 
# (Hint: Use the `AVG()` command)
avg_rating = pd.read_sql("books", engine)
avg_rating

,book_id,title,total_pages,rating,isbn,published_date,publisher_id
0,1,Lean Software Development: An Agile Toolkit,240,4.17,9780320000000,2003-05-18,5
1,2,Facing the Intelligence Explosion,91,3.87,None,2013-02-01,7
2,3,Scala in Action,419,3.74,9781940000000,2013-04-10,1
3,4,Patterns of Software: Tales from the Software ...,256,3.84,9780200000000,1996-08-15,1
4,5,Anatomy Of LISP,446,4.43,9780070000000,1978-01-01,3
5,6,Computing machinery and intelligence,24,4.17,None,2009-03-22,4
6,7,XML: Visual QuickStart Guide,269,3.66,9780320000000,2009-01-01,5
7,8,SQL Cookbook,595,3.95,9780600000000,2005-12-01,7
8,9,The Apollo Guidance Computer: Architecture And...,439,4.29,9781440000000,2010-07-01,6
9,10,Minds and Computers: An Introduction to the Ph...,222,3.54,9780750000000,2007-02-13,7


In [22]:
# 10. Write a SQL query to find the minimum book `rating` using the `books` table. 
# (Hint: Use the `MIN()` command)
min_rating = pd.read_sql("SELECT AVG(rating) FROM books;", engine)
min_rating

,avg
0,3.966


In [23]:
# 11. Write a SQL query to find the maximum book `rating` using the `books` table. 
# (Hint: Use the `MAX()` command)
max_rating = pd.read_sql("SELECT MAX(rating) FROM books", engine)
max_rating

,max
0,4.43


In [24]:
# 12. Write a SQL query to list the rows in the `books` table where `isbn` is not None. 
# (Hint: Use the `WHERE()` command)
non_null_isbn_rows = pd.read_sql("SELECT * FROM books WHERE isbn IS NOT NULL", engine)
non_null_isbn_rows

,book_id,title,total_pages,rating,isbn,published_date,publisher_id
0,1,Lean Software Development: An Agile Toolkit,240,4.17,9780320000000,2003-05-18,5
1,3,Scala in Action,419,3.74,9781940000000,2013-04-10,1
2,4,Patterns of Software: Tales from the Software ...,256,3.84,9780200000000,1996-08-15,1
3,5,Anatomy Of LISP,446,4.43,9780070000000,1978-01-01,3
4,7,XML: Visual QuickStart Guide,269,3.66,9780320000000,2009-01-01,5
5,8,SQL Cookbook,595,3.95,9780600000000,2005-12-01,7
6,9,The Apollo Guidance Computer: Architecture And...,439,4.29,9781440000000,2010-07-01,6
7,10,Minds and Computers: An Introduction to the Ph...,222,3.54,9780750000000,2007-02-13,7


In [25]:
# 13. Write a SQL query to list the rows in the `books` table where `total_pages` is greater than 400. 
# (Hint: Use the `WHERE()` command)
large_books = pd.read_sql("SELECT * FROM books WHERE total_pages > 400", engine)
large_books

,book_id,title,total_pages,rating,isbn,published_date,publisher_id
0,3,Scala in Action,419,3.74,9781940000000,2013-04-10,1
1,5,Anatomy Of LISP,446,4.43,9780070000000,1978-01-01,3
2,8,SQL Cookbook,595,3.95,9780600000000,2005-12-01,7
3,9,The Apollo Guidance Computer: Architecture And...,439,4.29,9781440000000,2010-07-01,6


In [26]:
# 14. Write a SQL query to list the rows in the `books` table where `published_date` is before 2000-01-01. 
# (Hint: Use the `WHERE()` command)
old_books = pd.read_sql("SELECT * FROM books WHERE published_date < '2000-01-01'", engine)
old_books

,book_id,title,total_pages,rating,isbn,published_date,publisher_id
0,4,Patterns of Software: Tales from the Software ...,256,3.84,9780200000000,1996-08-15,1
1,5,Anatomy Of LISP,446,4.43,9780070000000,1978-01-01,3


In [27]:
# 15. Write a SQL query to list the book titles along with the first name of the author. 
# (Hint: Several JOINs may be needed)

book_info = pd.read_sql(
    """
    SELECT books.title, authors.first_name
    FROM books
    JOIN book_authors ON books.book_id = book_authors.book_id
    JOIN authors ON authors.author_id = book_authors.author_id
    """,
    engine
)

book_info


,title,first_name
0,Lean Software Development: An Agile Toolkit,Merritt
1,Facing the Intelligence Explosion,Paul
2,Scala in Action,Yuval
3,Patterns of Software: Tales from the Software ...,Richard
4,Anatomy Of LISP,David
5,Computing machinery and intelligence,Anthony
6,XML: Visual QuickStart Guide,Alecos
7,SQL Cookbook,Linda
8,The Apollo Guidance Computer: Architecture And...,Anthony
9,Minds and Computers: An Introduction to the Ph...,Merritt


## How to clear the database

In [28]:
# Let's clear our database by dropping all our tables
with engine.begin() as connection:
    connection.execute(text("""
        DROP TABLE book_authors;

        DROP TABLE books;

        DROP TABLE authors;

        DROP TABLE publishers;
        """))

In [29]:
# Let's inspect out database again to confirm that the tables were deleted
inspector = inspect(engine)

# Get the table names
table_names = inspector.get_table_names()

# Print the table names
print(f"Existing tables in the database: {table_names}")

Existing tables in the database: []
